In [1]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
#
# iš originalių Zive įrašų sukuriami įrašai *.npy ir anotacijų failai *.json.

# Planas:
# sukuriame tuščią dataframe df_list_selected, iš kurio darysime naują list.json
# Nuskaitome iš bendros talpyklos list.json
# pasidarom sąrašą userId:recordId1, recordId2,.... 
# ciklas per visus pacientus userId:
#   suformuojame sąrašą recordId: ekstrasistolių skaičius ir surušiuojame pagal ekstrasistolių
#   skaičių mažėjančia tvarka
#   Surikiavus įrašus, įrašus atrenkame tokiu būdu:
# 		a. Atrenkame pirmiausiai tuos įrašus, kurie turi daugiausiai ekstrasistolių. 
#       Jei visi įrašai turi ekstrasistolių, tai atrenkame pirmus 10 (iš surikiuotų įrašų pagal ekstrasistolių kiekį).
#       Jei ne visi įrašai turi ekstrasistoles, tai atrenkame, tik tuos įrašus, kurie turi ekstrasistoles
#       (bet ne daugiau 10 įrašų), ir jei įrašų atrinkta ne daugiau 9, pridedame dar vieną įrašą be ekstrasistolių.
# 		b. Jei pacientas ekstrasistolių neturi, tai atrenkame ne daugiau vieno (pirmo) to paciento įrašo

#   Atrinkus įrašus, papildome df_list_selected  
# ciklo per pacientus pabaiga
# įrašome visus atrinkus įrašus į record_selected
# įrašome df_list_selected į list.json.
# išvedame į ekraną suvestinę ir df_list_selected

# SubjCode = userNr + recordingNr -- reikia taisyti /////////////////////////////////////////////////////

import shutil, sys
from icecream import ic
from pathlib import Path
import pandas as pd
import numpy as np

import json
from zive_util_vu import zive_read_file_1ch, create_dir, create_SubjCode


def get_userNr(rec_dir, userId):
    # Panaudodami df masyvą df_transl su įrašų numeriais iš ZIVE numerių gauname įrašų eilės numerius
    # Jei paciento Nr nėra - užvedamas įrašas
    
    # Patikriname, ar df_transl egzistuoja. Jei ne, sukuriame ir įrašome pirmą įraša
    file_path = Path(rec_dir, 'df_transl.csv')
    if (not file_path.exists()):
        # Paruošiame masyvą - žodyną numerių vertimui iš userId, registrationId į userNr, registrationNr ir atgal
        # ir įrašome į diską
        first_rec = {'userId':[userId], 'userNr':[1000] }
        df_transl = pd.DataFrame(first_rec)
        file_path = Path(rec_dir, 'df_transl.csv')
        df_transl.to_csv(file_path)
        return df_transl.loc[0, 'userNr']

    # Jei egzistuoja, nuskaitome vardų žodyną iš rec_dir aplanko
    file_path = Path(rec_dir, 'df_transl.csv')
    df_transl = pd.read_csv(file_path, index_col=0)
    # print(df_transl)
    # Ieškome, ar yra įrašas su userId
    # Jei userId nerandame, sukuriame naują įrašą su userId, userNr 
    if (df_transl.loc[(df_transl['userId'] == userId)]).empty:
        userNr = df_transl.loc[len(df_transl)-1, 'userNr'] + 1
        print("userNr=", userNr)
        new_row = {'userId':userId, 'userNr':userNr}
        df_transl = df_transl.append(new_row, ignore_index=True)
        file_path = Path(rec_dir, 'df_transl.csv')
        df_transl.to_csv(file_path)
        return userNr
    else:
        # Jei userId randame, gražiname userNr
        row = df_transl.loc[(df_transl['userId'] == userId)]
        return row['userNr'].values[0]


import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_VU'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

rec_dir = Path(db_path,'records_selected')

# Nuoroda į aplanką su transformuotu EKG duomenų rinkiniu
rec_dir_npy = Path(db_path, 'records_npy')

# Paliekamų anotacijų sąrašas
annot_list = ['N','S','V','U']

# Diskretizavimo dažnis
fs_zive = 200

print("\nSkriptas zive įrašų transformacijai\n")

print("Išeities duomenys skaitomi iš:", rec_dir)
print("Transformuoti duomenys rašomi į:", rec_dir_npy)
print("\n")


OS in my system :  win32

Skriptas zive įrašų transformacijai

Išeities duomenys skaitomi iš: C:\DI\Data\MIT&ZIVE\DUOM_VU\records_selected
Transformuoti duomenys rašomi į: C:\DI\Data\MIT&ZIVE\DUOM_VU\records_npy




In [2]:

# Sukūriame aplanką EKG sekų įrašymui
create_dir(rec_dir_npy)

pd.set_option("display.max_rows", 1000, "display.max_columns", 20)
pd.set_option('display.width', 2000)

# Nuskaitome Zive įrašų talpykloje laikomų įrašų sąrašą
file_path = Path(rec_dir, 'list.json')
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())

df_list = pd.json_normalize(data, record_path =['data'])

# print(df_list)
# Sugrupuojame įrašus į to paties paciento grupes
grouped = df_list.groupby(['userId','file_name'])
print(grouped)
print("\n")

SubjCodes = []

# Perrašome įrašus nauju formatu į kitą folderį
for key in grouped.groups:
    userId = key[0]
    file_name = str(key[1])
    
    userNr = get_userNr(rec_dir_npy, userId)
    SubjCode = create_SubjCode(userNr, file_name)  
    # print("SubjCode: ", SubjCode, userNr, file_name)

    SubjCodes.append(SubjCode)

    file_path = Path(rec_dir, file_name)
    signal = zive_read_file_1ch(file_path)  

    file_path = Path(rec_dir_npy, str(SubjCode) + '.npy')
    with open(file_path, 'wb') as f:
        np.save(f, signal)

    src = Path(rec_dir, file_name + '.json')
    dst = Path(rec_dir_npy, str(SubjCode) + '.json')
    shutil.copy2(src, dst)

print(SubjCodes)

info = {
    'db_folder':db_folder,
    'fs': fs_zive,
    'SubjCodes':SubjCodes,
    'annot_list':annot_list
    }

file_name = Path(rec_dir_npy,'info_create_z.json')
with open(file_name, 'w') as f:
    json.dump(info, f)
    
print("\nSchemos parametrai įrašyti į failą: ", file_name, "\n" )

Directory '/home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy' created successfully


userNr= 1001
userNr= 1002


/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)
/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)
/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)


userNr= 1003
userNr= 1004
userNr= 1005
userNr= 1006


/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)
/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)
/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)


userNr= 1007
userNr= 1008
userNr= 1009
userNr= 1010


...

userNr= 1044
userNr= 1045
userNr= 1046


/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)
/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)
/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)


userNr= 1047
userNr= 1048
userNr= 1049


/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)
/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)
/tmp/ipykernel_11311/1419352098.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transl = df_transl.append(new_row, ignore_index=True)


['1000.1630737.035', '1000.1630737.056', '1000.1630737.176', '1000.1630737.219', '1001.1630737.025', '1002.1630737.068', '1002.1630737.071', '1002.1630737.081', '1002.1630737.108', '1002.1630737.195', '1002.1630737.282', '1003.1630737.048', '1003.1630737.055', '1003.1630737.110', '1003.1630737.144', '1003.1630737.153', '1003.1630737.192', '1003.1630737.266', '1004.1630737.052', '1005.1630737.028', '1005.1630737.109', '1005.1630737.143', '1005.1630737.163', '1005.1630737.180', '1005.1630737.198', '1005.1630737.206', '1005.1630737.241', '1005.1630737.267', '1005.1630737.281', '1006.1630737.050', '1006.1630737.066', '1006.1630737.084', '1006.1630737.099', '1006.1630737.139', '1006.1630737.171', '1006.1630737.220', '1006.1630737.236', '1006.1630737.240', '1006.1630737.243', '1007.1630737.009', '1007.1630737.140', '1007.1630737.207', '1008.1630737.019', '1008.1630737.089', '1008.1630737.117', '1008.1630737.181', '1008.1630737.183', '1008.1630737.201', '1008.1630737.211', '1008.1630737.216',

In [3]:
# https://www.geeksforgeeks.org/python-shutil-copy2-method/#:~:text=shutil.copy
# 2%20%28%29%20method%20in%20Python%20is%20used%20to,destination%20can%20be%20a%20file%20or%20a%20directory.

# Susirandame anotacijų pasiskirstymą per pacientus ir pacientų skaičių
df_sum = df_list.groupby(['userId'],sort = False).sum()
# print(df_sum)
# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = df_list['userId'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(df_list)}')
print(f'\nĮrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
result.index.rename ('userID', inplace= True)
result.drop(labels=['incl'], axis=1, inplace=True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
print(result)

# Įrašome failų sąrašą į diską
file_path = Path(rec_dir_npy,'list.json')       
df_list.to_json(file_path, orient = 'table', index=False)
print(f'\nFailų sąrašas įrašytas:  {file_path}')
    


Viso pacientų: 50  EKG įrašų: 289

Įrašų pasiskirstymas per pacientus
                              N    S     V  U  flag  count
userID                                                    
61632b451e325518bb017bbd   5266    6    30  0     0      7
6190d4e63cd1d227c1303cea   5830   15     3  0     0     10
6144c588bd0cc52ba0275362   5280    0   413  0     0     10
61a8863e5cd547e980a4e970   5967  907   539  0     0     10
61632ace1e32557a62017bba   9887  833     5  0     0     10
6144c532bd0cc54c83275360    787    0     0  0     0      1
6143507abd0cc5051b275171   1885    5     4  0     0      3
61b32023cf0f347968cb1c91   2153    2     0  0     0      3
6144c682bd0cc5acb7275368   7718    0   982  0     0     10
61b31ff1cf0f34275ccb1c90   6950    9  1160  0     0     10
619b4bbbb481320f924e3da9   4489    1     6  0     0      7
616b00a811601579e9c2816c   5791    0   138  4     0      8
6144c4fbbd0cc552e427535f   6080    0  2275  0     0     10
617509686d4fb48519637743   1596   40     3  